In [41]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
# from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
# import numpy as np
# import sklearn
import pickle
from keras.applications import *
from keras.preprocessing.image import *
# importing dependencies
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import tqdm ##
from tqdm import * ##
# importing dependencies
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle

### Take label

In [64]:
label = pd.read_csv('data/phase1/mfcc/test/test_Y.csv',names=['ID','fname','label','verified'],header=0)
dicts_ = pickle.load(open('data/map.pkl','rb'))
label['trans']=label['label'].map(dicts_)
# label = label.drop(['ID','fname','verified','label'],axis=1)
label = label['trans'].values
print('label like:',label,'data#:', len(label))


label like: [13 34  7 ... 37 39  7] data#: 1895


### load all csv X data to big numpy array

In [65]:
folder = 'data/phase1/mfcc/stacking/'
files = os.listdir(folder)
# train_X=[]
# train_X=np.array(train_X)
for i,csv in enumerate(files):
    df = pd.read_csv(os.path.join(folder,csv),header=None)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
    df = df.drop(0,axis=1)
    #ASSERT df.iloc[5,0]==8652 ######################
#     print(df.values)
    if i==0:
        train_X = df.values
    else:
        train_X = np.concatenate((train_X,df.values),axis=1)
print('(ID#,prob_col#):',train_X.shape,' total files:',i+1)

(ID#,prob_col#): (1895, 738)  total files: 18


### load PCA

In [66]:
mfcc_pca = np.load('feature/mfcc/pca_rbf6/train_X_PCA.npy')
fbank_pca = np.load('feature/fbank/pca/train_X_PCA.npy')
raw_pca = np.load('feature/raw/pca/train_X_PCA.npy')
train = pd.read_csv("data/train_label.csv")
train_Y = pd.read_csv('data/phase1/mfcc/test/test_Y.csv',names=['ID','fname','label','verified'],header=0)
pcas = []
pcas.append(mfcc_pca)
pcas.append(fbank_pca)
pcas.append(raw_pca)

In [67]:

for i,npy in enumerate(pcas):
    npy = pd.DataFrame(npy)
    npy = pd.merge(train,npy, how='left',left_index=True,right_index=True)
    npy = pd.merge(train_Y,npy,how='inner',on='fname')
    npy = npy.drop(['ID','fname','label_x','verified','label_y','manually_verified'],axis=1)
    if i==0:
        pca_all = npy.values
    else:
        pca_all = np.concatenate((pca_all,npy.values),axis=1)
print('(ID#,prob_col#):',pca_all.shape,' total files:',i+1)
# mfcc_pca = pd.DataFrame(mfcc_pca)
# mfcc_pca = pd.merge(train,mfcc_pca, how='left',left_index=True,right_index=True)
# mfcc_pca = pd.merge(train_Y,mfcc_pca,how='inner',on='fname')
# mfcc_pca = mfcc_pca.drop(['ID','fname','label_x','verified','label_y','manually_verified'],axis=1)
# mfcc_pca

(ID#,prob_col#): (1895, 18)  total files: 3


### Concatenate all

In [68]:
train_X = np.concatenate((train_X,pca_all),axis=1)
train_Y = label
print(train_X.shape , train_Y.shape)

(1895, 756) (1895,)


In [47]:
def split_valid_set(X_all, Y_all, percentage):
    all_data_size = len(X_all)
    valid_data_size = int(floor(all_data_size * percentage))

    X_all, Y_all = _shuffle(X_all, Y_all)

    X_train, Y_train = X_all[0:valid_data_size], Y_all[0:valid_data_size]
    X_valid, Y_valid = X_all[valid_data_size:], Y_all[valid_data_size:]

    return X_train, Y_train, X_valid, Y_valid

In [48]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

In [49]:
# X_train, Y_train, X_valid, Y_valid = split_valid_set(train_X, train_Y, 0.5)

### XGB train

In [50]:
# def xgb_evaluate(min_child_weight,
#                  colsample_bytree,
#                  max_depth,
#                  subsample,
#                  gamma,
#                  alpha):

#     params['min_child_weight'] = int(min_child_weight)
#     params['cosample_bytree'] = max(min(colsample_bytree, 1), 0)
#     params['max_depth'] = int(max_depth)
#     params['subsample'] = max(min(subsample, 1), 0)
#     params['gamma'] = max(gamma, 0)
#     params['alpha'] = max(alpha, 0)


#     cv_result = xgb.cv(params, xgtrain, num_boost_round=num_rounds, nfold=5,
#              seed=random_state,
#              callbacks=[xgb.callback.early_stop(50)])

#     return -cv_result['test-mae-mean'].values[-1]

In [85]:
def train_xgb(max_delta_step,reg_lambda,reg_alpha,n_estimators,max_depth, subsample, min_child_weight, gamma, colsample_bytree, colsample_bylevel):
#, learning_rate
# def train_xgb(max_delta_step,reg_lambda,n_estimators,max_depth,min_child_weight,gamma):
    xgb_params = {
#         'tree_method':'gpu_hist',
#         'predictor':'gpu_predictor',
        'nthread': 12,
        'objective': 'multi:softmax',
        'num_class':41,
#         'n_estimators': int(750.6),
#         'eta': 0.01,
        'silent': 1,
#         'seed': 0,
        # for _train_internal
        'eval_metric': ['merror'],
        ######################
        'max_delta_step':int(max_delta_step),
        'reg_lambda': max(reg_lambda, 0),
        'reg_alpha': max(reg_alpha, 0),
        'n_estimators':int(n_estimators),
        'max_depth': int(max_depth),
        'subsample': max(min(subsample, 1), 0),
        'min_child_weight': int(min_child_weight),
#         'alpha': max(alpha, 0),
        'gamma': max(gamma, 0),
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'colsample_bylevel': max(min(colsample_bylevel, 1), 0),
#         'learning_rate': max(min(learning_rate, 1), 0),
    }
    # print(dTrain)
    score = xgb.cv(
        xgb_params, 
        dTrain, 
        num_boost_round=1500, #2000
        early_stopping_rounds=10, #100
        verbose_eval=False,
        maximize=False, #針對eval_metric
        nfold=3, #10
        seed=np.random.get_state()[1][0]
    )
    # logger.info(score)
    return -score['test-merror-mean'].iloc[-1] #回傳給BO去maximize的，只能改中間的值

In [96]:
def train():
#     X, Y, testX, testFid = prepare_data()
#     X, Y, testX = str2float(X, Y, testX)
#     X, testX = scale_data(X, testX)
    
#     if args.select_feature:
        # select feature
#         pass
    for iter in range(30): #the number of trying sampling，挑選幾個XGB存起來??
        np.random.seed(iter)
        logger.info('Iteration %2d, Current random seed: %2d' % (iter, np.random.get_state()[1][0]))
        # sampling data
        # trainX, trainY, validX, validY = sample_data(X, Y)
        trainX = train_X#pca_all #train_X#X_train
        trainY = train_Y#label#train_Y#Y_train
        testX = train_X#pca_all#train_X#X_valid
        testY = train_Y#label#train_Y#Y_valid
        global dTrain
        dTrain = xgb.DMatrix(trainX, label=trainY, nthread=12)
        global dTest
        dTest = xgb.DMatrix(testX, label=testY,nthread=12)

        # bayes_opt selection
        # parameter to be learning 
        logger.info('Setting parameters for BayesianOptimaization')       
        params = {
            'max_delta_step':(0,10),#(0,10), #0~10 0=>nolimit
            'reg_lambda': (25,60), #1e-5,200
            'reg_alpha': (0,1), #100
            'n_estimators':(365,410), #50~1600 #250~1200
            'max_depth': (1,16),#(1, 10), #65 #不能0=>no limit
            'subsample': (0.1, 1),
            'min_child_weight': (0, 1), #30
#             'alpha': (0, 10), #等同reg_alpha
            'gamma': (0,1), #30
            'colsample_bytree': (0.1, 1),
            'colsample_bylevel': (0.1, 1),
#             'learning_rate': (0.01, 1),
        }
        logger.info('Running BayesianOptimization')
        xgb_bayesopt = BayesianOptimization(train_xgb, params)
        xgb_bayesopt.maximize(init_points=10, n_iter=45) #10,50
        
        # get the best param
        best_params = xgb_bayesopt.res['max']['max_params']
        logger.info('Iteration: %d, XGBoost max auc: %f' % (iter, xgb_bayesopt.res['max']['max_val']))
        for param, val in best_params.items():
            logger.info('Param %s: %r' % (param, val))
        # setting xgboost param
        logger.info('Setting best parameters for BayesianOptimization')
        xgb_params = {
#             'tree_method':'gpu_hist',
#             'predictor':'gpu_predictor',
            'nthread': 12,
#             'n_estimators': int(750.6),
#             'eta': 0.01,
            'silent': 1,
            # for _train_internal
#             'eval_metric': ['map'],
            ######################
            'max_delta_step':int(best_params['max_delta_step']),
            'reg_lambda': max(best_params['reg_lambda'], 0),
            'reg_alpha': max(best_params['reg_alpha'], 0),
            'n_estimators':int(best_params['n_estimators']),
            'max_depth': int(best_params['max_depth']),
            'subsample': max(min(best_params['subsample'], 1), 0),
            'min_child_weight': int(best_params['min_child_weight']),
##             'alpha': max(best_params['alpha'], 0),
            'gamma': max(best_params['gamma'], 0),
            'colsample_bytree': max(min(best_params['colsample_bytree'], 1), 0),
            'colsample_bylevel': max(min(best_params['colsample_bylevel'], 1), 0),
#             'learning_rate': max(min(best_params['learning_rate'], 1), 0),
        }
        # training
        model = xgb.train(
            xgb_params, 
            dTrain, 
            num_boost_round=1500,  #2000
            verbose_eval=False, 
            maximize=False,
            
        )
        #save XGB best model
        writePickle(model, os.path.join('model', 'model_iter%d_%dfold_%f.pkl' % (iter, 3, xgb_bayesopt.res['max']['max_val'])))
        #n_fold
        # predict valid y
#         predY = model.predict(dTest)
#         result_df = pd.DataFrame(data={'y':predY})
#         joined_df = pd.DataFrame(testFid).join(result_df)


#         joined_df.to_csv(os.path.join('result', 'xgb_result%d_%dfold.csv' % (iter, 10)), index=False)


        

        # re-sorted the fid because of the random splitting data
        logger.info('----------------------------------------------------------------------\n\n\n')

In [ ]:
path = getPath()
dTrain = []
dTest = []
logger = Logger().getLogger()
train()

2018-05-11 08:23:33,589 - logHandler - train - INFO - Iteration  0, Current random seed:  0
2018-05-11 08:23:33,764 - logHandler - train - INFO - Setting parameters for BayesianOptimaization
2018-05-11 08:23:33,775 - logHandler - train - INFO - Running BayesianOptimization
Initialization
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bylevel |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   n_estimators |   reg_alpha |   reg_lambda |   subsample | 
    1 | 01m21s |   -0.29183 |              0.1001 |             0.3996 |    0.5925 |           5.2868 |      3.1866 |             0.5239 |       365.9276 |      0.8173 |      30.7833 |      0.7717 | 
    2 | 01m36s |   -0.27495 |              0.8262 |             0.3483 |    0.0530 |           5.7273 |      8.2

   34 | 01m59s |   -0.27916 |              0.1000 |             1.0000 |    1.0000 |           5.9101 |      4.2166 |             1.0000 |       408.9284 |      0.0000 |      30.3262 |      1.0000 | 
   35 | 01m54s |   -0.29446 |              1.0000 |             1.0000 |    0.0000 |           0.0000 |      1.0000 |             0.0000 |       374.0913 |      1.0000 |      25.0000 |      1.0000 | 
   36 | 13m16s |   -0.30608 |              1.0000 |             1.0000 |    0.0000 |           7.6771 |      8.3067 |             0.0000 |       405.2843 |      1.0000 |      30.3161 |      1.0000 | 
   37 | 06m55s |   -0.29444 |              1.0000 |             1.0000 |    1.0000 |          10.0000 |     16.0000 |             1.0000 |       410.0000 |      1.0000 |      25.0000 |      1.0000 | 


/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-2.97121333e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 2, 'warnflag': 2}
  " state: %s" % convergence_dict)


   38 | 01m20s |   -0.27810 |              0.1000 |             1.0000 |    1.0000 |          10.0000 |      1.0000 |             0.0000 |       365.0000 |      0.0000 |      25.0000 |      1.0000 | 
   39 | 02m06s |   -0.30607 |              0.1000 |             0.1000 |    1.0000 |          10.0000 |     16.0000 |             0.0000 |       382.4868 |      0.0000 |      25.0000 |      1.0000 | 
   40 | 02m31s |   -0.28496 |              1.0000 |             0.1000 |    1.0000 |           0.0000 |     16.0000 |             1.0000 |       365.0000 |      1.0000 |      44.0258 |      1.0000 | 


/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-6.94508828e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 58, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


...

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([4.20638903e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


2018-05-11 11:09:08,468 - logHandler - train - INFO - Iteration: 0, XGBoost max auc: -0.271243
2018-05-11 11:09:08,470 - logHandler - train - INFO - Param max_delta_step: 10.0
2018-05-11 11:09:08,471 - logHandler - train - INFO - Param reg_lambda: 40.26914775764784
2018-05-11 11:09:08,471 - logHandler - train - INFO - Param reg_alpha: 7.746043489979467e-15
2018-05-11 11:09:08,472 - logHandler - train - INFO - Param n_estimators: 400.130917400757
2018-05-11 11:09:08,474 - logHandler - train - INFO - Param max_depth: 15.999999999999956
2018-05-11 11:09:08,475 - logHandler - train - INFO - Param subsample: 1.0
2018-05-11 11:09:08,476 - logHandler - train - INFO - Param min_child_weight: 0.0
2018-05-11 11:09:08,477 - logHandler - train - INFO - Param gamma: 0.631530020073976
2018-05-11 11:09:08,478 - logHandler - train - INFO - Param colsample_bytree: 0.999999999999993
2018-05-11 11:09:08,479 - logHandler - train - INFO - Param colsample_bylevel: 0.1
2018-05-11 11:09:08,479 - logHandler - 

   27 | 02m57s |   -0.28759 |              1.0000 |             1.0000 |    0.0000 |          10.0000 |      1.0000 |             0.0000 |       365.0000 |      1.0000 |      25.0000 |      1.0000 | 
   28 | 14m49s |   -0.30817 |              1.0000 |             1.0000 |    0.0000 |           0.0000 |     16.0000 |             0.0000 |       410.0000 |      1.0000 |      60.0000 |      1.0000 | 
   29 | 00m30s |   -0.28601 |              1.0000 |             0.1000 |    0.0000 |          10.0000 |      1.0000 |             0.0000 |       365.0000 |      0.0000 |      45.0612 |      1.0000 | 
   30 | 01m38s |   -0.31662 |              1.0000 |             1.0000 |    0.0000 |          10.0000 |     16.0000 |             0.0000 |       365.0000 |      0.0000 |      60.0000 |      1.0000 | 
   31 | 00m39s |   -0.28603 |              1.0000 |             1.0000 |    0.0000 |           0.0000 |      1.0000 |             0.0000 |       376.3523 |      1.0000 |      39.2013 |      1.0000 | 


/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.35095344e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)


   52 | 03m28s |   -0.27071 |              1.0000 |             0.1000 |    0.0000 |          10.0000 |      7.0453 |             1.0000 |       386.1785 |      1.0000 |      49.6394 |      1.0000 | 
   53 | 02m36s |   -0.29761 |              1.0000 |             0.1000 |    0.0000 |           0.0000 |     16.0000 |             0.0000 |       396.4354 |      1.0000 |      25.0000 |      1.0000 | 
   54 | 02m22s |   -0.27968 |              1.0000 |             1.0000 |    0.0000 |           0.0000 |      1.0000 |             1.0000 |       376.6043 |      1.0000 |      25.0000 |      1.0000 | 
   55 | 01m53s |   -0.26861 |              1.0000 |             0.1000 |    0.0000 |          10.0000 |      1.0000 |             0.0000 |       410.0000 |      1.0000 |      37.1313 |      1.0000 | 
2018-05-11 13:39:19,315 - logHandler - train - INFO - Iteration: 1, XGBoost max auc: -0.267527
2018-05-11 13:39:19,316 - logHandler - train - INFO - Param max_delta_step: 10.0
2018-05-11 13:39:19,316 

In [44]:
X_valid.shape , Y_valid.shape, X_train.shape , Y_train.shape, dTrain.get_weight()

((948, 264), (948,), (947, 264), (947,), array([], dtype=float32))

In [27]:
xgb1 = xgb.XGBClassifier(tree_method='gpu_hist',predictor='gpu_predictor',objective='multi:softmax')
xgb1.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob',
       predictor='gpu_predictor', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1, tree_method='gpu_hist')

In [28]:
xgb1.score(X_valid,Y_valid)

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6455696202531646

***

In [59]:
train = pd.read_csv("data/train_label.csv")
train

,fname,label,manually_verified
0,00044347.wav,Hi-hat,0
1,001ca53d.wav,Saxophone,1
2,002d256b.wav,Trumpet,0
3,0033e230.wav,Glockenspiel,1
4,00353774.wav,Cello,1
5,003b91e8.wav,Cello,0
6,003da8e5.wav,Knock,1
7,0048fd00.wav,Gunshot_or_gunfire,1
8,004ad66f.wav,Clarinet,0
9,0063ab88.wav,Computer_keyboard,0


In [60]:
mfcc_pca = pd.DataFrame(mfcc_pca)
mfcc_pca = pd.merge(train,mfcc_pca, how='left',left_index=True,right_index=True)
mfcc_pca

,fname,label,manually_verified,0,1,2,3,4,5
0,00044347.wav,Hi-hat,0,-0.092997,0.054736,0.105257,0.030881,0.043051,0.065048
1,001ca53d.wav,Saxophone,1,-0.194650,0.046794,0.067786,-0.024712,0.036319,0.073897
2,002d256b.wav,Trumpet,0,0.176952,0.049333,0.013577,-0.030624,0.017623,-0.009100
3,0033e230.wav,Glockenspiel,1,0.153554,0.084642,0.031795,0.011289,-0.012398,0.015777
4,00353774.wav,Cello,1,-0.209155,-0.038918,-0.040240,0.122628,0.079755,0.043273
5,003b91e8.wav,Cello,0,-0.290912,0.086930,-0.166344,-0.062807,-0.002551,0.043976
6,003da8e5.wav,Knock,1,0.161126,0.022441,-0.009888,-0.014193,0.019312,-0.000979
7,0048fd00.wav,Gunshot_or_gunfire,1,0.150040,-0.063527,-0.037651,-0.067343,0.106101,-0.028000
8,004ad66f.wav,Clarinet,0,-0.059628,0.043215,0.155853,-0.001425,0.014030,0.042836
9,0063ab88.wav,Computer_keyboard,0,0.005038,0.086374,0.032094,-0.026018,-0.054419,-0.081478


In [57]:
train_Y = pd.read_csv('data/phase1/mfcc/test/test_Y.csv',names=['ID','fname','label','verified'],header=0)
train_Y

,ID,fname,label,verified
0,5695,9b3dfe43.wav,Tearing,0
1,1058,1d26c5e5.wav,Electric_piano,1
2,4862,82f9184f.wav,Clarinet,0
3,124,03a01285.wav,Knock,1
4,4269,7390ae7a.wav,Burping_or_eructation,1
5,8652,e9c76647.wav,Tambourine,0
6,3945,6a64d4a4.wav,Oboe,0
7,4347,75b6cbcb.wav,Oboe,0
8,5775,9d868658.wav,Laughter,1
9,1702,2eeac352.wav,Fireworks,0


In [25]:
train = pd.read_csv("data/train_label.csv")
test = pd.read_csv("data/sample_submission.csv")
label = pd.read_csv('data/phase1/mfcc/test/test_Y.csv',names=['ID','fname','label','ver'],header=0)
mike1 = pd.read_csv('data/phase1/mfcc/stacking/mike_predict_1.csv')
mike1 = pd.merge(mike1,label,on='ID',how='inner')
mike2 = pd.read_csv('data/phase1/mfcc/stacking/mike_predict_2.csv')
mike2 = pd.merge(mike2,label,on='ID',how='inner')
mike3 = pd.read_csv('data/phase1/mfcc/stacking/mike_predict_3.csv')
mike3 = pd.merge(mike3,label,on='ID',how='inner')
mike4 = pd.read_csv('data/phase1/mfcc/stacking/mike_predict_4.csv')
mike4 = pd.merge(mike4,label,on='ID',how='inner')
mike5 = pd.read_csv('data/phase1/mfcc/stacking/mike_predict_5.csv')
mike5 = pd.merge(mike5,label,on='ID',how='inner')
mike6 = pd.read_csv('data/phase1/mfcc/stacking/mike_predict_6.csv')
mike6 = pd.merge(mike6,label,on='ID',how='inner')

In [26]:
mike1

,ID,0,1,2,3,4,5,6,7,8,...,34,35,36,37,38,39,40,fname,label,ver
0,5695,2.310457e-01,0.000034,7.460250e-03,9.418055e-06,4.032949e-05,1.120928e-02,2.320042e-02,8.579953e-05,2.471905e-02,...,0.000063,0.000019,1.728770e-03,1.577650e-02,0.000141,0.000741,6.522915e-05,9b3dfe43.wav,Tearing,0
1,1058,1.517785e-04,0.003879,5.391893e-04,1.392749e-01,5.643258e-04,1.150002e-03,1.148319e-05,7.920492e-03,2.776606e-03,...,0.643624,0.039170,1.420293e-04,1.346285e-04,0.029799,0.009291,2.125310e-05,1d26c5e5.wav,Electric_piano,1
2,4862,8.027842e-05,0.003299,2.144003e-05,2.948551e-04,1.800109e-05,1.459022e-05,7.920331e-06,4.639407e-01,4.448483e-06,...,0.001586,0.000305,3.456194e-07,7.725090e-06,0.000032,0.001596,1.447491e-05,82f9184f.wav,Clarinet,0
3,124,6.930202e-04,0.000768,1.258271e-03,3.208297e-04,2.538871e-03,1.276575e-01,1.047017e-03,3.362295e-04,1.433433e-04,...,0.041530,0.001502,1.814795e-04,1.086782e-05,0.007584,0.001962,1.674535e-04,03a01285.wav,Knock,1
4,4269,1.184819e-04,0.000072,8.157063e-06,1.105321e-04,6.713203e-06,1.207649e-04,8.152965e-06,3.891747e-05,8.818816e-05,...,0.000039,0.001695,4.252597e-05,1.183774e-06,0.000161,0.001214,7.261047e-05,7390ae7a.wav,Burping_or_eructation,1
5,8652,2.236898e-03,0.000027,3.530817e-05,2.070725e-03,2.248686e-05,1.692215e-05,5.840601e-05,7.562254e-06,6.139342e-05,...,0.000433,0.000135,3.581619e-06,5.527779e-06,0.000896,0.000234,2.737718e-04,e9c76647.wav,Tambourine,0
6,3945,1.156638e-05,0.001133,1.097198e-03,1.023563e-05,4.162205e-03,1.987280e-04,4.627917e-07,2.711397e-03,1.615423e-05,...,0.000599,0.000405,4.998346e-06,3.731492e-07,0.003154,0.000130,1.521391e-05,6a64d4a4.wav,Oboe,0
7,4347,1.930442e-05,0.003496,4.623220e-03,5.523527e-05,1.956414e-04,8.769006e-04,7.185360e-05,4.722181e-03,4.891935e-05,...,0.000586,0.004558,3.661737e-06,4.170156e-05,0.000066,0.000387,2.872082e-05,75b6cbcb.wav,Oboe,0
8,5775,6.181363e-04,0.001252,2.185558e-03,3.058583e-03,1.101774e-03,3.688487e-03,8.227184e-04,2.209437e-03,2.826831e-03,...,0.000033,0.020439,1.640526e-02,7.494623e-04,0.000446,0.000398,2.189038e-03,9d868658.wav,Laughter,1
9,1702,1.245969e-02,0.000051,7.500663e-04,2.575303e-04,7.618025e-04,1.861568e-02,3.936417e-02,9.350741e-05,1.696403e-02,...,0.000641,0.000425,2.600361e-02,5.164795e-02,0.000340,0.000243,4.088125e-04,2eeac352.wav,Fireworks,0


In [11]:
kk = train.iloc[8652,1]
kk

'Tambourine'

In [28]:
# mike1 = pd.merge(mike1,label,on='ID',how='inner')

mike2 = mike2.drop(['ID','fname','ver'],axis=1)
mike1

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,label
0,2.310457e-01,0.000034,7.460250e-03,9.418055e-06,4.032949e-05,1.120928e-02,2.320042e-02,8.579953e-05,2.471905e-02,1.206903e-02,...,0.004203,0.001651,0.000063,0.000019,1.728770e-03,1.577650e-02,0.000141,0.000741,6.522915e-05,Tearing
1,1.517785e-04,0.003879,5.391893e-04,1.392749e-01,5.643258e-04,1.150002e-03,1.148319e-05,7.920492e-03,2.776606e-03,6.123833e-04,...,0.000781,0.000569,0.643624,0.039170,1.420293e-04,1.346285e-04,0.029799,0.009291,2.125310e-05,Electric_piano
2,8.027842e-05,0.003299,2.144003e-05,2.948551e-04,1.800109e-05,1.459022e-05,7.920331e-06,4.639407e-01,4.448483e-06,8.925809e-07,...,0.000020,0.000211,0.001586,0.000305,3.456194e-07,7.725090e-06,0.000032,0.001596,1.447491e-05,Clarinet
3,6.930202e-04,0.000768,1.258271e-03,3.208297e-04,2.538871e-03,1.276575e-01,1.047017e-03,3.362295e-04,1.433433e-04,7.383644e-05,...,0.000411,0.001216,0.041530,0.001502,1.814795e-04,1.086782e-05,0.007584,0.001962,1.674535e-04,Knock
4,1.184819e-04,0.000072,8.157063e-06,1.105321e-04,6.713203e-06,1.207649e-04,8.152965e-06,3.891747e-05,8.818816e-05,7.656837e-05,...,0.000866,0.003301,0.000039,0.001695,4.252597e-05,1.183774e-06,0.000161,0.001214,7.261047e-05,Burping_or_eructation
5,2.236898e-03,0.000027,3.530817e-05,2.070725e-03,2.248686e-05,1.692215e-05,5.840601e-05,7.562254e-06,6.139342e-05,1.101737e-02,...,0.928692,0.000171,0.000433,0.000135,3.581619e-06,5.527779e-06,0.000896,0.000234,2.737718e-04,Tambourine
6,1.156638e-05,0.001133,1.097198e-03,1.023563e-05,4.162205e-03,1.987280e-04,4.627917e-07,2.711397e-03,1.615423e-05,1.296195e-05,...,0.000001,0.000028,0.000599,0.000405,4.998346e-06,3.731492e-07,0.003154,0.000130,1.521391e-05,Oboe
7,1.930442e-05,0.003496,4.623220e-03,5.523527e-05,1.956414e-04,8.769006e-04,7.185360e-05,4.722181e-03,4.891935e-05,1.543390e-06,...,0.000012,0.000518,0.000586,0.004558,3.661737e-06,4.170156e-05,0.000066,0.000387,2.872082e-05,Oboe
8,6.181363e-04,0.001252,2.185558e-03,3.058583e-03,1.101774e-03,3.688487e-03,8.227184e-04,2.209437e-03,2.826831e-03,1.344062e-03,...,0.000138,0.002262,0.000033,0.020439,1.640526e-02,7.494623e-04,0.000446,0.000398,2.189038e-03,Laughter
9,1.245969e-02,0.000051,7.500663e-04,2.575303e-04,7.618025e-04,1.861568e-02,3.936417e-02,9.350741e-05,1.696403e-02,1.167972e-02,...,0.001295,0.000215,0.000641,0.000425,2.600361e-02,5.164795e-02,0.000340,0.000243,4.088125e-04,Fireworks


In [34]:
dicts_ = pickle.load(open('data/map.pkl','rb'))
dicts_

{'Hi-hat': 0,
 'Saxophone': 1,
 'Trumpet': 2,
 'Glockenspiel': 3,
 'Cello': 4,
 'Knock': 5,
 'Gunshot_or_gunfire': 6,
 'Clarinet': 7,
 'Computer_keyboard': 8,
 'Keys_jangling': 9,
 'Snare_drum': 10,
 'Writing': 11,
 'Laughter': 12,
 'Tearing': 13,
 'Fart': 14,
 'Oboe': 15,
 'Flute': 16,
 'Cough': 17,
 'Telephone': 18,
 'Bark': 19,
 'Chime': 20,
 'Bass_drum': 21,
 'Bus': 22,
 'Squeak': 23,
 'Scissors': 24,
 'Harmonica': 25,
 'Gong': 26,
 'Microwave_oven': 27,
 'Burping_or_eructation': 28,
 'Double_bass': 29,
 'Shatter': 30,
 'Fireworks': 31,
 'Tambourine': 32,
 'Cowbell': 33,
 'Electric_piano': 34,
 'Meow': 35,
 'Drawer_open_or_close': 36,
 'Applause': 37,
 'Acoustic_guitar': 38,
 'Violin_or_fiddle': 39,
 'Finger_snapping': 40}

In [35]:
# mike2 = mike2.drop(['ID','fname','ver'],axis=1)
mike1['trans']=mike1['label'].map(dicts_)
label = list(mike1['trans'])
label

[13,
 34,
 7,
 5,
 28,
 32,
 15,
 15,
 12,
 31,
 23,
 30,
 4,
 24,
 33,
 10,
 38,
 37,
 23,
 5,
 16,
 38,
 13,
 16,
 29,
 39,
 12,
 9,
 17,
 28,
 11,
 32,
 2,
 14,
 11,
 20,
 26,
 39,
 29,
 32,
 25,
 33,
 13,
 31,
 28,
 0,
 38,
 0,
 29,
 4,
 13,
 30,
 29,
 38,
 2,
 7,
 1,
 10,
 17,
 12,
 23,
 26,
 38,
 14,
 38,
 35,
 31,
 27,
 15,
 4,
 19,
 15,
 35,
 31,
 34,
 3,
 37,
 9,
 17,
 31,
 10,
 20,
 5,
 12,
 30,
 32,
 0,
 11,
 29,
 2,
 4,
 10,
 0,
 4,
 19,
 30,
 22,
 4,
 1,
 0,
 16,
 16,
 1,
 14,
 40,
 13,
 28,
 14,
 15,
 4,
 34,
 21,
 16,
 2,
 6,
 16,
 7,
 2,
 26,
 19,
 28,
 11,
 25,
 16,
 12,
 29,
 37,
 38,
 24,
 1,
 34,
 16,
 38,
 21,
 34,
 35,
 10,
 14,
 27,
 21,
 21,
 12,
 33,
 6,
 1,
 23,
 5,
 2,
 23,
 39,
 21,
 8,
 7,
 3,
 33,
 16,
 3,
 34,
 11,
 12,
 36,
 20,
 35,
 38,
 19,
 1,
 13,
 30,
 7,
 20,
 34,
 16,
 12,
 10,
 21,
 38,
 25,
 19,
 23,
 30,
 4,
 33,
 25,
 12,
 26,
 13,
 26,
 12,
 31,
 0,
 10,
 21,
 9,
 40,
 27,
 22,
 19,
 29,
 26,
 31,
 17,
 15,
 16,
 26,
 38,
 38,
 4,
 27,
 12,


In [38]:
mike1 = mike1.drop(['label','trans'],axis=1)
mike1 = np.array(mike1)
mike1 = mike1.values
mike1

array([[2.31045710e-01, 3.35746600e-05, 7.46024960e-03, ...,
        1.41390450e-04, 7.41442260e-04, 6.52291500e-05],
       [1.51778510e-04, 3.87878620e-03, 5.39189300e-04, ...,
        2.97994700e-02, 9.29132600e-03, 2.12531010e-05],
       [8.02784200e-05, 3.29891130e-03, 2.14400320e-05, ...,
        3.21515840e-05, 1.59612880e-03, 1.44749065e-05],
       ...,
       [2.82808440e-04, 1.87970120e-03, 2.61777370e-04, ...,
        1.13384480e-04, 6.38530840e-05, 4.21484040e-05],
       [2.92850300e-05, 6.68284800e-04, 1.85118250e-03, ...,
        3.34779320e-05, 9.84250070e-01, 2.41563980e-06],
       [5.60485900e-07, 2.72542200e-04, 6.42650000e-05, ...,
        2.48254770e-04, 1.01912700e-04, 3.03670000e-05]])

In [20]:
df = pd.read_csv('data/phase1/mfcc/stacking/mike_predict_1.csv',header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,5695,2.310457e-01,0.000034,7.460250e-03,9.418055e-06,4.032949e-05,1.120928e-02,2.320042e-02,8.579953e-05,2.471905e-02,...,1.922805e-04,0.004203,0.001651,0.000063,0.000019,1.728770e-03,1.577650e-02,0.000141,0.000741,6.522915e-05
1,1058,1.517785e-04,0.003879,5.391893e-04,1.392749e-01,5.643258e-04,1.150002e-03,1.148319e-05,7.920492e-03,2.776606e-03,...,1.336942e-04,0.000781,0.000569,0.643624,0.039170,1.420293e-04,1.346285e-04,0.029799,0.009291,2.125310e-05
2,4862,8.027842e-05,0.003299,2.144003e-05,2.948551e-04,1.800109e-05,1.459022e-05,7.920331e-06,4.639407e-01,4.448483e-06,...,1.456362e-04,0.000020,0.000211,0.001586,0.000305,3.456194e-07,7.725090e-06,0.000032,0.001596,1.447491e-05
3,124,6.930202e-04,0.000768,1.258271e-03,3.208297e-04,2.538871e-03,1.276575e-01,1.047017e-03,3.362295e-04,1.433433e-04,...,2.280328e-04,0.000411,0.001216,0.041530,0.001502,1.814795e-04,1.086782e-05,0.007584,0.001962,1.674535e-04
4,4269,1.184819e-04,0.000072,8.157063e-06,1.105321e-04,6.713203e-06,1.207649e-04,8.152965e-06,3.891747e-05,8.818816e-05,...,2.163198e-05,0.000866,0.003301,0.000039,0.001695,4.252597e-05,1.183774e-06,0.000161,0.001214,7.261047e-05
5,8652,2.236898e-03,0.000027,3.530817e-05,2.070725e-03,2.248686e-05,1.692215e-05,5.840601e-05,7.562254e-06,6.139342e-05,...,4.521946e-06,0.928692,0.000171,0.000433,0.000135,3.581619e-06,5.527779e-06,0.000896,0.000234,2.737718e-04
6,3945,1.156638e-05,0.001133,1.097198e-03,1.023563e-05,4.162205e-03,1.987280e-04,4.627917e-07,2.711397e-03,1.615423e-05,...,8.837261e-06,0.000001,0.000028,0.000599,0.000405,4.998346e-06,3.731492e-07,0.003154,0.000130,1.521391e-05
7,4347,1.930442e-05,0.003496,4.623220e-03,5.523527e-05,1.956414e-04,8.769006e-04,7.185360e-05,4.722181e-03,4.891935e-05,...,1.008416e-04,0.000012,0.000518,0.000586,0.004558,3.661737e-06,4.170156e-05,0.000066,0.000387,2.872082e-05
8,5775,6.181363e-04,0.001252,2.185558e-03,3.058583e-03,1.101774e-03,3.688487e-03,8.227184e-04,2.209437e-03,2.826831e-03,...,2.838157e-02,0.000138,0.002262,0.000033,0.020439,1.640526e-02,7.494623e-04,0.000446,0.000398,2.189038e-03
9,1702,1.245969e-02,0.000051,7.500663e-04,2.575303e-04,7.618025e-04,1.861568e-02,3.936417e-02,9.350741e-05,1.696403e-02,...,5.078173e-01,0.001295,0.000215,0.000641,0.000425,2.600361e-02,5.164795e-02,0.000340,0.000243,4.088125e-04


In [28]:
df = pd.read_csv('data/phase1/mfcc/stacking/mow_predict_1.csv',header=None)
if df.iloc[0,0] == 'fname':
    df = df.drop(0,axis=0)
df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
1,9b3dfe43.wav,1.345092e-04,1.301059e-06,1.655290e-05,3.100925e-05,1.261864e-07,6.707866e-04,1.353832e-04,3.628572e-07,1.502499e-01,...,5.280791e-05,7.732307e-05,1.337797e-05,5.099835e-06,2.716369e-07,8.322137e-04,5.500814e-03,6.604541e-05,1.225595e-07,1.064936e-04
2,1d26c5e5.wav,2.450175e-04,1.181947e-05,6.718703e-05,2.347599e-01,3.899182e-05,1.140950e-02,8.271242e-05,3.257621e-05,1.321871e-03,...,1.286891e-03,8.027336e-05,1.774957e-03,5.984293e-01,1.160985e-03,3.739293e-05,7.961946e-06,1.447119e-03,1.669340e-04,4.750869e-04
3,82f9184f.wav,6.074694e-10,2.807165e-06,5.821687e-10,7.198089e-09,6.730634e-09,4.073515e-13,3.723949e-13,9.972614e-01,2.537792e-14,...,1.658404e-15,2.794465e-13,2.530564e-08,1.534512e-06,1.481852e-13,3.767440e-18,3.291789e-12,2.232611e-07,8.806542e-07,1.277226e-15
4,03a01285.wav,1.443713e-09,1.176412e-11,2.914073e-09,8.799134e-11,1.135180e-07,9.990612e-01,2.663566e-06,7.432923e-12,1.654219e-08,...,6.705965e-07,4.348645e-09,8.331401e-09,9.367019e-08,1.970713e-11,3.747513e-10,1.676837e-12,2.715842e-09,1.933715e-09,2.466821e-08
5,7390ae7a.wav,2.937305e-10,3.872118e-12,2.790983e-12,9.081368e-11,4.037929e-10,2.545254e-09,1.325479e-13,8.183093e-11,6.945775e-11,...,1.328805e-10,3.864008e-09,1.040601e-08,1.211714e-12,1.906311e-08,3.186559e-11,3.424198e-12,3.146658e-09,1.113679e-09,2.517648e-11
6,e9c76647.wav,7.786378e-06,3.225282e-10,3.344255e-08,3.803266e-08,4.906596e-09,4.158570e-09,3.101323e-09,9.506107e-09,2.847463e-10,...,8.993581e-12,9.999669e-01,3.719828e-07,5.729349e-09,3.435429e-11,1.763415e-11,3.385243e-09,6.286717e-08,3.550825e-07,5.983674e-08
7,6a64d4a4.wav,6.157673e-10,3.359246e-06,1.031203e-05,5.514756e-10,1.354734e-06,4.974269e-13,1.195879e-11,6.099683e-06,3.572953e-15,...,4.695789e-15,1.620733e-08,5.896035e-10,5.292503e-07,1.887938e-08,2.979874e-16,1.701670e-10,9.215284e-09,2.590872e-04,2.449347e-15
8,75b6cbcb.wav,2.454610e-07,9.089721e-05,5.811114e-06,1.793679e-06,1.190902e-07,1.707035e-09,5.241724e-10,1.052218e-04,2.910166e-11,...,7.599212e-12,2.012114e-07,6.403960e-06,4.655032e-04,4.225509e-07,6.373224e-15,3.394217e-09,5.468773e-07,2.268435e-03,3.360427e-11
9,9d868658.wav,3.364268e-06,5.308573e-06,1.364735e-06,2.039646e-06,9.373528e-04,5.380822e-03,5.479405e-06,6.353830e-07,1.889912e-02,...,1.291192e-03,7.805792e-04,3.667865e-05,5.938084e-06,5.911153e-05,3.263826e-05,5.659563e-05,2.986188e-05,3.911962e-04,1.595545e-05
10,2eeac352.wav,5.009679e-04,1.083224e-06,3.351288e-04,1.639683e-06,1.865459e-04,2.355602e-03,8.681491e-03,2.702939e-08,4.202268e-07,...,9.640036e-01,1.229296e-04,6.849977e-05,2.107110e-06,3.690365e-07,4.735893e-05,1.050612e-05,9.628999e-07,1.148766e-04,1.009101e-06
